In [1]:
#Research references:
#1) Dry/wet cough classification: https://link.springer.com/article/10.1007/s10439-013-0741-6
#2) Pneumonia classification: https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=6987276
#3) https://espace.library.uq.edu.au/data/UQ_344963/s41943203_phd_submission.pdf?Expires=1585601065&Key-Pair-Id=APKAJKNBJ4MJBJNC6NLQ&Signature=Lnpf6wT8rkozSh9av7U9nGuC7WAH6KuI2Cj3Y7G366gkGlh8D-Ie1Kc~TyBAUu~uMsVltleJcSv3p6TCm6HdFnhpyoTgLcYh6eFfvQwIUqbk1Bf4JZldgB~BDKUOwY1G0pA-HoKjvIAu3avO98SMO35upakm9OEBByd4nC9aXsjKRThd6bTpq1qIuuD9gh1l5FaM6hNRB0c2lCf4Q3adx7C3FW0NMwdWhcuF45A9f~dO3zTWWSQamoo5Otc-PHMMt96TetNcML~jy9ghgJeCPY6DJLUIwQAt03fENBluS~TjTJ17WD~n51xiRofb94fEJHoRHh0d-430LLwr7BX4IA__

In [2]:
import numpy as np
import os
#import pywt #wavelets
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.utils import mediainfo
from pydub.playback import play
import matplotlib.pyplot as plt
#import seaborn as sn
import python_speech_features as spe_feats
import pandas as pd
from scipy.stats import kurtosis, skew
from scipy.signal import lfilter
import librosa
import pysptk
import math
import sys

#settings
import config

## Reading recordings

In [3]:
all_s=[]
all_label=[]
all_id=[]
all_fs=[]

In [4]:
#Read wav data set

if config.featExtr_skip is False:
    print("Readings wavs...")

    #only list files in FOLDER_PATH directory
    wav_files = [f for f in os.listdir(config.FOLDER_PATH) if os.path.isfile(os.path.join(config.FOLDER_PATH, f))]
    for file_name in wav_files:
    
        fname_noExt = os.path.splitext(file_name)[0] #file name without extension
    
        #full path file name
        full_fname = config.FOLDER_PATH+file_name
        print(full_fname)
    
        # load audio
        s = AudioSegment.from_wav(full_fname)
        print(full_fname)
        all_s.append(s)
        #sampling rate:
        info = mediainfo(full_fname)
        fs = float(info['sample_rate'])
        all_fs.append(fs)
    
        #get ID of recording
        ID = fname_noExt.split('-')[-2] #for the current type of naming
        #print(file_name)
        #print(ID)
        all_id.append(ID)
    
        #get label
        label = fname_noExt.split('-')[-1] #for the current type of naming
        #print(label)
        all_label.append(label)

Readings wavs...
data/YT_set/edited_wavs/edit_Spring Allergy Coughing-7Ez5Wc_esBg-Dry.wav
data/YT_set/edited_wavs/edit_Spring Allergy Coughing-7Ez5Wc_esBg-Dry.wav
data/YT_set/edited_wavs/edit_Coughing 51-LkxvBb2VXbs-Dry.wav
data/YT_set/edited_wavs/edit_Coughing 51-LkxvBb2VXbs-Dry.wav
data/YT_set/edited_wavs/edit_Wet coughing-0QQxKN-KC1U-Wet.wav
data/YT_set/edited_wavs/edit_Wet coughing-0QQxKN-KC1U-Wet.wav
data/YT_set/edited_wavs/edit_Dry Coughing Fit in the Afternoon.-A5s2ZgwQ1VM-Dry.wav
data/YT_set/edited_wavs/edit_Dry Coughing Fit in the Afternoon.-A5s2ZgwQ1VM-Dry.wav
data/YT_set/edited_wavs/edit_Coughing 77-2Mw-s5jnqXU-Wet.wav
data/YT_set/edited_wavs/edit_Coughing 77-2Mw-s5jnqXU-Wet.wav
data/YT_set/edited_wavs/edit_Wheezing Chest and Wet Cough 2-5905FxXz9dI-Wet.wav
data/YT_set/edited_wavs/edit_Wheezing Chest and Wet Cough 2-5905FxXz9dI-Wet.wav
data/YT_set/edited_wavs/edit_Coughing 46-dg-I9j76-t8-Wet.wav
data/YT_set/edited_wavs/edit_Coughing 46-dg-I9j76-t8-Wet.wav
data/YT_set/edited_

Listening to some of the audios

In [5]:
if config.featExtr_skip is False:
    np.where(np.array(all_label)=='Dry')

In [6]:
if config.featExtr_skip is False:
    np.where(np.array(all_label)=='Wet')

In [7]:
if config.featExtr_skip is False:
    s=all_s[15]
    s

## Feature extraction

In [8]:
import featureExtractionFunctions as featExtrLib

if config.featExtr_skip is False:

    feats = featExtrLib.feature_extraction_Step(all_s,all_id,all_label)
    
       

High-pass filtering...
Computing features...


ValueError: attempt to get argmax of an empty sequence

## Load  (or store) features 

In [ ]:
feats_fname = 'feats_df.pkl'

if config.featExtr_skip is False:
    #Store feature df
    feats.to_pickle(feats_fname)
else:
    #Load feature df
    feats = pd.read_pickle(feats_fname)

## Pre-processing of features

In [ ]:
feats2 = featExtrLib.processingNaNvalues(feats)

In [ ]:
label_dict = featExtrLib.createLabelDict_addLabel2df(feats2)
mean_std_feats = featExtrLib.frame_mean_std_chunk_modeling (feats2,label_dict)

## Model training

In [ ]:
X_train = mean_std_feats.drop(['label','Id','subIdx'], 1).copy()
y_train =  mean_std_feats['label'].copy()

ID_train = mean_std_feats['Id']

#ID_train.size

In [ ]:
import modelTrainingFunctions as modelTrainLib

pred_probs = modelTrainLib.modelTraining(X_train,y_train,ID_train)

In [ ]:
mean_pred_probs = modelTrainLib.get_predClass_per_audio(pred_probs, label_dict)

## Evaluation

In [ ]:
import classifEvaluationFunctions as evalLib
evalLib.evaluation_Step(mean_pred_probs)
    